## Filter results - find top containers with no location
The Aspace API is mosly built around getting and editing objects by thier IDs. In order to do a bulk operation, you'll need to have some way to arrive at a list of item IDs. You get all IDs for a type of object in a repository, all child IDs from a known Item, or if you have time, look at every item.

### Example, Get a sample of top contianers
First, we'll need to log in again:

In [43]:
# Log in again

# we need our request making tool, which we 
# can get by importing it like so:
import json
import requests

# first, we'll save bits of connection information (on the right)
# as variables (on the right)

USER = 'ian'
PASS = 'aspacewhospace'
HOST = 'http://localhost:8089'

# Here's our authentication function. It will return a session
# token if it works, or the value False if if doesn't


def aspace_auth(host, username, password):
    auth = requests.post(host + '/users/' + username + '/login',
                        params={'password' : password})
    if auth.status_code == 200:
        token = auth.json()['session']
        headers = {'X-ArchivesSpace-Session': token}
        return(headers)
    else:
        return(False)

headers = aspace_auth(HOST, USER, PASS)
print(headers)

{'X-ArchivesSpace-Session': '7451d18e847f2d91c8feff595a41ff923cd5d2a7ec11b25863e92640012d4615'}


### API reference: archival object children
https://archivesspace.github.io/archivesspace/api/#post-repositories-repo_id-top_containers

You can view the documentation for the API call for getting all top level containers from a repoitory above. I find the documentation a bit confusing here, but you must include some parameter to tell the API how many IDs you want (the results are paginated)

In [54]:
# The format for the API call is/repositories/:repo_id/top_containers.
# You can fill in the details fo the call below.

container_req = requests.get(HOST +'/repositories/2/top_containers?all_ids=True',
                             headers=headers)

In [55]:
containers = container_req.json()


In [56]:
len(containers)


29948

In [58]:
# OK looks like we got a list of all the IDs
# lets look at a the first 10 to see what they look like.
containers[0:10]

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [59]:
# If you needed to inspect all of your top level containers
# for example to look for a problem, you might end up working in batches
# Let's do one batch here. I'll make it a bit smaller.
container_sample_ids = containers[0:500]

Now, we can use a loop to do something with the first 1000 ids. Lets return to the documentation and look at the API call for getting the data that describes a top level container: https://archivesspace.github.io/archivesspace/api/#get-repositories-repo_id-top_containers-id

In [72]:
# We'll use a loop to look get the first 1000
# I'm going to start with an empty list and add
# each record to the list in the loop.

container_sample = []

for identifier in container_sample_ids:
    container_req = requests.get(HOST + '/repositories/2/top_containers/'
                                 + str(identifier),
                                headers=headers)
    container_sample.append(container_req.json())

In [82]:
# Let's look at a sample
container_sample[3]

{'active_restrictions': [],
 'collection': [{'display_string': 'MS2079 American Association of University Professors records',
   'identifier': '2007.057',
   'ref': '/repositories/2/accessions/251'}],
 'container_locations': [{'jsonmodel_type': 'container_location',
   'ref': '/locations/246',
   'start_date': '2016-07-05',
   'status': 'current',
   'system_mtime': '2017-10-17 12:28:47 UTC',
   'user_mtime': '2017-10-17 12:28:47 UTC'}],
 'create_time': '2017-10-17T12:28:47Z',
 'created_by': 'admin',
 'display_string': 'Item 2007.057',
 'indicator': '2007.057',
 'jsonmodel_type': 'top_container',
 'last_modified_by': 'admin',
 'lock_version': 1,
 'long_display_string': '2007.057, Item 2007.057',
 'repository': {'ref': '/repositories/2'},
 'restricted': False,
 'series': [],
 'system_mtime': '2017-10-17T12:28:47Z',
 'type': 'item',
 'uri': '/repositories/2/top_containers/4',
 'user_mtime': '2017-10-17T12:28:47Z'}

In [106]:
# OK container location is represented as a list
# below, we'll add the empty containers to a new
# list called empty_locations

# start with an empty list
empty_locations = []

# for each container in our sample of containers
for container in container_sample:
    # if the value of the continaer_locations key is an empty list
    if container['container_locations'] == []:
        # add it to our empty list
        empty_locations.append(container)

In [107]:
# check the length of our list
len(empty_locations)

265

In [108]:
empty_locations

[{'active_restrictions': [],
  'collection': [{'display_string': 'Americans for UNESCO',
    'identifier': '2016--117',
    'ref': '/repositories/2/accessions/1152'}],
  'container_locations': [],
  'create_time': '2017-10-17T12:28:57Z',
  'created_by': 'admin',
  'display_string': 'Box 7',
  'indicator': '7',
  'jsonmodel_type': 'top_container',
  'last_modified_by': 'admin',
  'lock_version': 1,
  'long_display_string': '2016--117, Box 7',
  'repository': {'ref': '/repositories/2'},
  'restricted': False,
  'series': [],
  'system_mtime': '2017-10-17T12:28:57Z',
  'type': 'Box',
  'uri': '/repositories/2/top_containers/282',
  'user_mtime': '2017-10-17T12:28:57Z'},
 {'active_restrictions': [],
  'collection': [{'display_string': 'National Association of Elementary School Principals',
    'identifier': '2017--036',
    'ref': '/repositories/2/accessions/1201'}],
  'container_locations': [],
  'create_time': '2017-10-17T12:28:59Z',
  'created_by': 'admin',
  'display_string': 'Box 1-35

### Putting it all together

So now we've done some exploration, let's put it all together in once little script

In [105]:
# import libraries

import requests
import json

# first, we'll save bits of connection information (on the right)
# as variables (on the right)

USER = 'ian'
PASS = 'aspacewhospace'
HOST = 'http://localhost:8089'

# Here's our authentication function. It will return a session
# token if it works, or the value False if if doesn't


def aspace_auth(host, username, password):
    auth = requests.post(host + '/users/' + username + '/login',
                        params={'password' : password})
    if auth.status_code == 200:
        token = auth.json()['session']
        headers = {'X-ArchivesSpace-Session': token}
        return(headers)
    else:
        return(False)

headers = aspace_auth(HOST, USER, PASS)

# OK, now get a list of all the top containers
container_req = requests.get(HOST +'/repositories/2/top_containers?page=9&page_size=250',
                             headers=headers)
# assign the results to a new variable (for readabilty)
containers = container_req.json()['results']


#get all the container_ids
for container in containers:
    if container['container_locations'] == []:
        # Do something!
        # update a field and send back to Aspace?
        # write to a file?
        # Delete the container?
        print(container['uri'])

{'X-ArchivesSpace-Session': '6875d07b086b26a54b919d3108fadec2b7f6f3896a56dc09d4c0637a68aabce5'}
/repositories/2/top_containers/2596
/repositories/2/top_containers/2597
/repositories/2/top_containers/2598
/repositories/2/top_containers/2599
/repositories/2/top_containers/2600
/repositories/2/top_containers/2601
/repositories/2/top_containers/2602
/repositories/2/top_containers/2603
/repositories/2/top_containers/2604
/repositories/2/top_containers/2605
/repositories/2/top_containers/2606
/repositories/2/top_containers/2607
/repositories/2/top_containers/2608
/repositories/2/top_containers/2609
/repositories/2/top_containers/2610
/repositories/2/top_containers/2611
/repositories/2/top_containers/2612
/repositories/2/top_containers/2613
/repositories/2/top_containers/2614
/repositories/2/top_containers/2615
/repositories/2/top_containers/2616
/repositories/2/top_containers/2617
/repositories/2/top_containers/2618
/repositories/2/top_containers/2619
/repositories/2/top_containers/2620
/rep